In [1]:
import numpy as np
import pandas as pd

In [4]:
text="""Students: S1(101), S2(102), S3(103)
Companies:
CompanyA (30min, 2 rounds, 2 panels)
CompanyB (45min, 1 round, 1 panel)
Shortlisting:
S1: A,B; S2: A; S3: B
Slot: 9 AM - 12 PM"""
readlines=text.split('\n')
readlines

['Students: S1(101), S2(102), S3(103)',
 'Companies:',
 'CompanyA (30min, 2 rounds, 2 panels)',
 'CompanyB (45min, 1 round, 1 panel)',
 'Shortlisting:',
 'S1: A,B; S2: A; S3: B',
 'Slot: 9 AM - 12 PM']

In [5]:
stopping_time={'A1': [('S1(101)', 0, 30),
  ('S1(101)', 30, 60),
  ('S2(102)', 60, 90),
  ('S2(102)', 90, 120)],
 'A2': [],
 'B1': [('S1(101)', 60, 105), ('S3(103)', 0, 45)],
 'S1(101)': [('A1', 0, 30), ('A1', 30, 60), ('B1', 60, 105)],
 'S2(102)': [('A1', 60, 90), ('A1', 90, 120)],
 'S3(103)': [('B1', 0, 45)]}


In [ ]:
from math import gcd
from functools import reduce
text="""Students: S1(101), S2(102), S3(103)
Companies:
CompanyA (30min, 2 rounds, 2 panels)
CompanyB (45min, 1 round, 1 panel)
Shortlisting:
S1: A,B; S2: A; S3: B
Slot: 9 AM - 12 PM"""
readlines=text.split('\n')
readlines
students=readlines[0].split(':')[1].split(',')
students=[s.strip() for s in students]
companies=[]
flag=True
i=2
while flag:
    if 'compan' in readlines[i].lower():
        name=readlines[i].split('(')[0].strip().replace('Company','')
        time=int(readlines[i].split('(')[1].split(',')[0].replace('min','').strip())
        rounds=int(readlines[i].split('(')[1].split(',')[1].split()[0].strip())
        panels=int(readlines[i].split('(')[1].split(',')[2].strip().split(')')[0].split()[0].strip())
        companies.append({"names":name, "time":time, "rounds":rounds, "panels":panels})
        i+=1
    else:
        flag=False
time_values = [company['time'] for company in companies]
gcd_time = reduce(gcd, time_values)

shortlist=readlines[i+1].split(';')
shortlist_dict={}
for s in shortlist:
    target_student, companiess = s.split(':')
   
    student = next((s for s in students if target_student.strip() in s), target_student).strip()
    companiess = [c.strip() for c in companiess.split(',')]
    shortlist_dict[student] = companiess
slot=readlines[i+2].split(':')[1].strip().split('-')

start_time_str=slot[0].strip()
end_time_str=slot[1].strip()
from datetime import datetime
# Convert to datetime objects
start_time = datetime.strptime(start_time_str, "%I %p")
end_time = datetime.strptime(end_time_str, "%I %p")

# Compute difference in minutes
time_difference = end_time - start_time
minutes = time_difference.total_seconds() / 60

minutes
stopping_time={}
for company in companies:
    for j in range(company["panels"]):
        stopping_time[company["names"] + str(j + 1)] = []
for student in students:
    stopping_time[student] = []

schedules = {}
for company in companies:
    for panel in range(company["panels"]):
        panel_name = company["names"] + str(panel + 1)
        schedules[panel_name] = []
        for start in range(0, int(minutes), gcd_time):
            end = start + company["time"]
            if end <= minutes:
                schedules[panel_name].append((start, end))
# print(schedules)
# Initialize the matrix with zeros
matrix = [[0 for _ in range(len(students))] for _ in range(len(companies))]

# Populate the matrix
for i, company in enumerate(companies):
    for j, student in enumerate(students):
        if company['names'] in shortlist_dict.get(student, []):
            matrix[i][j] = company['rounds']

conflicts = []
def compute_waiting_time(matrix, companies, students, stopping_time):
        total_waiting = 0
        day_start = 0
        day_end = 180  # 9AM-12PM in minutes

        # Student waiting time calculation
        for j, student in enumerate(students):
            # Get all scheduled interviews across all companies
            all_interviews = []
            for entry in stopping_time.get(student, []):
                company_name = entry[0].rstrip('123')  # Remove panel number
                all_interviews.append( (entry[1], entry[2], company_name) )

            # Sort by start time
            all_interviews.sort(key=lambda x: x[0])

            # Create timeline of occupied periods
            timeline = []
            for start, end, _ in all_interviews:
                timeline.append( (start, end) )

            # Merge overlapping intervals
            merged = []
            for start, end in sorted(timeline):
                if not merged or merged[-1][1] < start:
                    merged.append( [start, end] )
                else:
                    merged[-1][1] = max(merged[-1][1], end)

            # Calculate gaps considering the whole day
            gaps = []
            prev_end = day_start
            for start, end in merged:
                if start > prev_end:
                    gaps.append( (prev_end, start) )
                prev_end = max(prev_end, end)
            if prev_end < day_end:
                gaps.append( (prev_end, day_end) )

            # Get remaining companies for this student
            remaining_companies = [companies[i] for i in range(len(companies)) 
                                if matrix[i][j] > 0]

            # Check gaps against remaining companies
            for gap_start, gap_end in gaps:
                gap_duration = gap_end - gap_start
                if gap_duration <= 0:
                    continue

                # Find if any remaining company could fit in this gap
                can_utilize = any(
                    comp['time'] <= gap_duration 
                    for comp in remaining_companies
                )

                if not can_utilize:
                    total_waiting += gap_duration

        # Company waiting time calculation
        for i, company in enumerate(companies):
            # Check if company has completed all rounds with all students
            if not all(matrix[i][j] == 0 for j in range(len(students))):
                continue

            # Collect all time slots from all panels
            company_slots = []
            for panel in range(1, company['panels'] + 1):
                panel_name = company['names'] + str(panel)
                for entry in stopping_time.get(panel_name, []):
                    company_slots.append( (entry[1], entry[2]) )

            # Sort and merge intervals
            company_slots.sort()
            merged = []
            for start, end in company_slots:
                if not merged or merged[-1][1] < start:
                    merged.append( [start, end] )
                else:
                    merged[-1][1] = max(merged[-1][1], end)

            # Calculate company's timeline gaps
            gaps = []
            prev_end = day_start
            for start, end in merged:
                if start > prev_end:
                    gaps.append( (prev_end, start) )
                prev_end = max(prev_end, end)
            if prev_end < day_end:
                gaps.append( (prev_end, day_end) )

            # Add all company gaps to waiting time
            total_waiting += sum(end - start for start, end in gaps)

        return total_waiting
    
    
def backtrack_schedules(matrix, companies, students, stopping_time, schedules, waiting_time,best_schedule):
    """
    Backtracks to find a valid schedule for interviews between companies and students, minimizing waiting time.
    Args:
        matrix (list of list of int): A 2D list where matrix[i][j] represents the number of interview rounds 
                                      remaining between company i and student j.
        companies (list of dict): A list of dictionaries where each dictionary contains information about a company, 
                                  including its name and the number of panels available for interviews.
                                  Example: [{"names": "CompanyA", "panels": 2}, {"names": "CompanyB", "panels": 1}]
        students (list of str): A list of student names corresponding to the columns of the matrix.
        stopping_time (dict): A dictionary where keys are panel names or student names, and values are lists of tuples 
                              representing the scheduled interview slots. Each tuple is of the form (name, start, end).
        schedules (dict): A dictionary where keys are panel names, and values are lists of available time slots 
                          for the respective panels. Each time slot is represented as a tuple (start, end).
    Returns:
        int or None: The total waiting time if a valid schedule is found, or None if no valid schedule exists.
    Notes:
        - The function uses recursion to explore all possible schedules.
        - If no valid slot is found for a student and company, the function backtracks to try other possibilities.
        - Waiting time is calculated based on the gaps between consecutive interview slots for each panel.
    """
    # with open("matrix_log.txt", "a") as file:
    #     file.write("Current Matrix State:\n")
    #     for row in matrix:
    #         file.write(f"{row}\n")
    #     file.write("\n")
    # Base case: if all companies have no rounds left for all students
    current_waiting = compute_waiting_time(matrix, companies, students, stopping_time)
    if current_waiting >= best_schedule["waiting_time"]:
        return
    if all(all(x == 0 for x in row) for row in matrix):
        total_waiting = compute_waiting_time(matrix, companies, students, stopping_time)
        if total_waiting < best_schedule["waiting_time"]:
            best_schedule["waiting_time"] = total_waiting
            best_schedule["stopping_time"] = copy.deepcopy(stopping_time)
            with open("schedule_log.txt", "a") as file:
                file.write(f"New Best Waiting Time: {total_waiting}\n")
                for k, v in stopping_time.items():
                    file.write(f"{k}: {v}\n")
                file.write("\n")
        return
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] > 0:  # If there are rounds left for the student and company
                for k in range(companies[i]["panels"]):
                    #store student completed slots
                    # Check if there are available slots for the company panels
                    
                    panel_name = companies[i]["names"] + str(k + 1)
                    available_slots = schedules[panel_name]
                    if available_slots:  # If there are available slots for the panel
                    
                        # Iterate through all possible slots for the panel
                        for slot in available_slots[:]:
                            start, end = slot
                            
                            # Check if the student is available during this slot
                            if all((end <= s[1] and start< s[1]) or (start >= s[2] and end>s[2]) for s in stopping_time[students[j]]) and all((end <= s[1] and start< s[1]) or (start >= s[2] and end>s[2]) for s in stopping_time[panel_name]) and 0 <= start < 180 and 0 < end <= 180:
                                matrix[i][j] -= 1 
                                # Assign this slot
                                
                                schedules[panel_name].remove(slot)  # Remove the slot only once
                                # Remove overlapping slots
                                removed = [s for s in schedules[panel_name] if (s[0] < end) and (s[1] > start)]
                                student_removed = []
                                
                                for l in range(len(matrix[0])):
                                        for s in schedules[panel_name]:
                                                for st in stopping_time[students[l]]:
                                                        if (((s[0] >= st[1] and s[0] < st[2]) or (s[1] > st[1] and s[1] <= st[2]) ) and st[0] == panel_name):
                                                            student_removed.append(s)
                                                            # print("student removed", s,st)
                                                            
                                # Combine both removed and student_removed
                                combined_removed = list(set(removed + student_removed))
                                
                                # Remove slots from the schedule
                                schedules[panel_name] = [s for s in schedules[panel_name] if s not in combined_removed]
                                # # Update the schedules , matrix and stopping times
                                # schedules[panel_name] = [s for s in schedules[panel_name] if not (s[0] >= start and s[0] < end)]
                                # print(slot, combined_removed, stopping_time[students[j]])
                                stopping_time[panel_name].append((students[j], start, end))
                                stopping_time[students[j]].append((panel_name, start, end))
                                # print(matrix,panel_name,students[j], stopping_time[students[j]])
                                # If we found a valid schedule, return the waiting time
                                total_waiting = compute_waiting_time(matrix, companies, students, stopping_time)
                                if total_waiting < best_schedule["waiting_time"]:
                                    backtrack_schedules(matrix, companies, students, stopping_time, schedules, waiting_time, best_schedule)
                                
                                stopping_time[panel_name].pop()
                                stopping_time[students[j]].pop()
                                matrix[i][j] += 1
                                schedules[panel_name].append(slot)  # Add the slot back at the end
                                # Add the removed overlapping slots back
                                schedules[panel_name].extend(combined_removed)
                                schedules[panel_name].sort()  # Ensure the slots are sorted
                                # with open("matrix_log.txt", "a") as file:
                                #     file.write(f"Backtracked: Restored slot {slot} for {students[j]} in {panel_name}\n")
                else:
                    best_schedule["conflicts"].append((companies[i], students[j], matrix[i][j]))
                                
                                
    return




import copy



best_schedule = {"waiting_time": float('inf'), "stopping_time": {},"conflicts": []}
# Initialize stopping_time with deep empty dicts like your original code
from copy import deepcopy
stopping_time_copy = deepcopy(stopping_time)

backtrack_schedules(matrix, companies, students, stopping_time_copy, schedules, 0, best_schedule)

print("Minimum Waiting Time:", best_schedule["waiting_time"])
print("Final Stopping Times:")
for k, v in best_schedule["stopping_time"].items():
    print(k, v)


KeyboardInterrupt: 

In [2]:
best_schedule

{'waiting_time': 480,
 'stopping_time': {'A1': [('S1(101)', 0, 30),
   ('S1(101)', 30, 60),
   ('S2(102)', 60, 90),
   ('S2(102)', 90, 120)],
  'A2': [],
  'B1': [('S1(101)', 60, 105), ('S3(103)', 0, 45)],
  'S1(101)': [('A1', 0, 30), ('A1', 30, 60), ('B1', 60, 105)],
  'S2(102)': [('A1', 60, 90), ('A1', 90, 120)],
  'S3(103)': [('B1', 0, 45)]},
 'conflicts': [({'names': 'B', 'time': 45, 'rounds': 1, 'panels': 1},
   'S3(103)',
   1),
  ({'names': 'B', 'time': 45, 'rounds': 1, 'panels': 1}, 'S3(103)', 1),
  ({'names': 'B', 'time': 45, 'rounds': 1, 'panels': 1}, 'S3(103)', 1),
  ({'names': 'B', 'time': 45, 'rounds': 1, 'panels': 1}, 'S3(103)', 1),
  ({'names': 'B', 'time': 45, 'rounds': 1, 'panels': 1}, 'S3(103)', 1),
  ({'names': 'B', 'time': 45, 'rounds': 1, 'panels': 1}, 'S3(103)', 1),
  ({'names': 'B', 'time': 45, 'rounds': 1, 'panels': 1}, 'S1(101)', 1),
  ({'names': 'B', 'time': 45, 'rounds': 1, 'panels': 1}, 'S1(101)', 1),
  ({'names': 'B', 'time': 45, 'rounds': 1, 'panels': 1}, 

In [1]:
from math import gcd
from functools import reduce
import copy
from datetime import datetime, timedelta

# Input parsing
text = """Students: S1(101), S2(102), S3(103)
Companies:
CompanyA (30min, 2 rounds, 2 panels)
CompanyB (45min, 1 round, 1 panel)
Shortlisting:
S1: A,B; S2: A; S3: B
Slot: 9 AM - 12 PM"""

readlines = text.split('\n')
students = [s.strip() for s in readlines[0].split(':')[1].split(',')]
companies = []
i = 2
while i < len(readlines) and 'compan' in readlines[i].lower():
    name = readlines[i].split('(')[0].strip().replace('Company', '')
    time = int(readlines[i].split('(')[1].split(',')[0].replace('min', '').strip())
    rounds = int(readlines[i].split('(')[1].split(',')[1].split()[0].strip())
    panels = int(readlines[i].split('(')[1].split(',')[2].strip().split(')')[0].split()[0].strip())
    companies.append({"names": name, "time": time, "rounds": rounds, "panels": panels})
    i += 1

time_values = [company['time'] for company in companies]
gcd_time = reduce(gcd, time_values)

shortlist = readlines[i+1].split(';')
shortlist_dict = {}
for s in shortlist:
    target_student, companiess = s.split(':')
    student = next((s for s in students if target_student.strip() in s), target_student).strip()
    companiess = [c.strip() for c in companiess.split(',')]
    shortlist_dict[student] = companiess

slot = readlines[i+2].split(':')[1].strip().split('-')
start_time = datetime.strptime(slot[0].strip(), "%I %p")
end_time = datetime.strptime(slot[1].strip(), "%I %p")
minutes = (end_time - start_time).total_seconds() / 60

# Initialize schedules and stopping times
stopping_time = {company["names"] + str(j + 1): [] for company in companies for j in range(company["panels"])}
for student in students:
    stopping_time[student] = []

schedules = {}
for company in companies:
    for panel in range(company["panels"]):
        panel_name = company["names"] + str(panel + 1)
        schedules[panel_name] = [(start, start + company["time"]) for start in range(0, int(minutes), gcd_time) if start + company["time"] <= minutes]

# Initialize matrix
matrix = [[0 for _ in range(len(students))] for _ in range(len(companies))]
for i, company in enumerate(companies):
    for j, student in enumerate(students):
        if company['names'] in shortlist_dict.get(student, []):
            matrix[i][j] = company['rounds']

# Compute waiting time
def compute_waiting_time(matrix, companies, students, stopping_time):
    total_waiting = 0
    day_start, day_end = 0, int(minutes)

    # Student waiting time
    for j, student in enumerate(students):
        interviews = [(entry[1], entry[2], entry[0].rstrip('123')) for entry in stopping_time.get(student, [])]
        interviews.sort(key=lambda x: x[0])
        merged = []
        for start, end, _ in interviews:
            if not merged or merged[-1][1] < start:
                merged.append([start, end])
            else:
                merged[-1][1] = max(merged[-1][1], end)
        
        gaps = []
        prev_end = day_start
        for start, end in merged:
            if start > prev_end:
                gaps.append((prev_end, start))
            prev_end = max(prev_end, end)
        if prev_end < day_end:
            gaps.append((prev_end, day_end))

        remaining_companies = [companies[i] for i in range(len(companies)) if matrix[i][j] > 0]
        for gap_start, gap_end in gaps:
            gap_duration = gap_end - gap_start
            if gap_duration <= 0:
                continue
            if not any(comp['time'] <= gap_duration for comp in remaining_companies):
                total_waiting += gap_duration

    # Company waiting time
    for i, company in enumerate(companies):
        if not all(matrix[i][j] == 0 for j in range(len(students))):
            continue
        company_slots = []
        for panel in range(1, company['panels'] + 1):
            panel_name = company['names'] + str(panel)
            company_slots.extend((entry[1], entry[2]) for entry in stopping_time.get(panel_name, []))
        
        company_slots.sort()
        merged = []
        for start, end in company_slots:
            if not merged or merged[-1][1] < start:
                merged.append([start, end])
            else:
                merged[-1][1] = max(merged[-1][1], end)
        
        gaps = []
        prev_end = day_start
        for start, end in merged:
            if start > prev_end:
                gaps.append((prev_end, start))
            prev_end = max(prev_end, end)
        if prev_end < day_end:
            gaps.append((prev_end, day_end))
        total_waiting += sum(end - start for start, end in gaps)

    return total_waiting

# Convert minutes to time string
def minutes_to_time(minutes, base_time):
    time = base_time + timedelta(minutes=minutes)
    return time.strftime("%I:%M %p").lstrip('0')

# Backtracking algorithm
def backtrack_schedules(matrix, companies, students, stopping_time, schedules, best_schedule):
    # Prune if current waiting time is worse than the best found
    current_waiting = compute_waiting_time(matrix, companies, students, stopping_time)
    if current_waiting >= best_schedule["waiting_time"]:
        return

    # Base case: all rounds scheduled
    if all(all(x == 0 for x in row) for row in matrix):
        total_waiting = compute_waiting_time(matrix, companies, students, stopping_time)
        if total_waiting < best_schedule["waiting_time"]:
            best_schedule["waiting_time"] = total_waiting
            best_schedule["stopping_time"] = copy.deepcopy(stopping_time)
        return

    # Prioritize students with the most remaining interviews
    student_loads = [(sum(row[j] for row in matrix), j) for j in range(len(students))]
    student_loads.sort(reverse=True)  # Sort by number of remaining rounds
    for _, j in student_loads:
        for i in range(len(companies)):
            if matrix[i][j] > 0:
                company = companies[i]
                for k in range(company["panels"]):
                    panel_name = company["names"] + str(k + 1)
                    for slot in schedules[panel_name][:]:
                        start, end = slot
                        # Corrected slot availability check
                        if all(end <= s[1] or start >= s[2] for s in stopping_time[students[j]]) and \
                           all(end <= s[1] or start >= s[2] for s in stopping_time[panel_name]) and \
                           0 <= start < minutes and end <= minutes:
                            # Schedule the interview
                            matrix[i][j] -= 1
                            schedules[panel_name].remove(slot)
                            stopping_time[panel_name].append((students[j], start, end))
                            stopping_time[students[j]].append((panel_name, start, end))

                            backtrack_schedules(matrix, companies, students, stopping_time, schedules, best_schedule)

                            # Backtrack
                            stopping_time[students[j]].pop()
                            stopping_time[panel_name].pop()
                            schedules[panel_name].append(slot)
                            schedules[panel_name].sort()
                            matrix[i][j] += 1
                # If no slot is found, record conflict
                if matrix[i][j] > 0:
                    best_schedule["conflicts"].append((company["names"], students[j], matrix[i][j]))
                return  # Process one student-company pair at a time



# Run backtracking
best_schedule = {"waiting_time": float('inf'), "stopping_time": {}, "conflicts": []}
stopping_time_copy = copy.deepcopy(stopping_time)
backtrack_schedules(matrix, companies, students, stopping_time_copy, schedules, best_schedule)

# Print results
print("Minimum Waiting Time:", best_schedule["waiting_time"])
if best_schedule["conflicts"]:
    print("Conflicts (unscheduled interviews):")
    for company, student, rounds in best_schedule["conflicts"]:
        print(f"{student} with Company{company}: {rounds} round(s)")
print_schedule(best_schedule["stopping_time"], students, start_time)

SyntaxError: unterminated string literal (detected at line 186) (1167791608.py, line 186)